In [ ]:
%matplotlib inline

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

# Loading
df = pd.read_csv(Path("../data/raw/delivery_history.csv"))

# Factor Visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1. Distance and Rain Dependency
sns.scatterplot(data=df, x='distance_km', y='delivery_time_min', hue='is_rainy', alpha=0.5, ax=axes[0])
axes[0].set_title("Distance vs Time (with Rain Consideration)")

# 2. Rush Hour Impact
sns.lineplot(data=df, x='hour', y='delivery_time_min', ax=axes[1], color='red', marker='o')
axes[1].set_title("Average Delivery Time by Hour of Day")
axes[1].axvspan(17, 19, color='yellow', alpha=0.3, label='Rush Hour')
axes[1].legend()

plt.show()